In [25]:
from sklearn.datasets import load_iris
dataset =load_iris()
X=dataset.data
y=dataset.target
#print(dataset.DESCR)


In [8]:
import numpy as np
attribute_means=X.mean(axis=0)
X_d=np.array(X>=attribute_means,dtype='int')    #bool value, the logistic 
X_d[:5]    #'numpy.ndarray' object has no attribute 'head()'

array([[0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

In [16]:
from collections import defaultdict
from operator import itemgetter
def train_feature_value(X,y_true,feature_index,value):
    class_counts=defaultdict(int)
    for sample,y in zip(X,y_true):    #zip : when zip is needed？     for sample,y in (X,y_true)?
        if sample[feature_index]==value:
            class_counts[y]+=1
    sorted_class_counts=sorted(class_counts.items(),key=itemgetter(1),reverse=True)
    most_frequent_class=sorted_class_counts[0][0]
    
    incorrect_predictions=[class_count for class_value,class_count in class_counts.items() if class_value!=most_frequent_class]
    error= sum(incorrect_predictions)
    return most_frequent_class,error
def train_on_feature(X,y_true,feature_index):
    values=set(X[:,feature_index])
    predictors={}
    errors=[]
    for current_value in values:
        most_frequent_class,error=train_feature_value(X,y_true,feature_index,current_value)
        predictors[current_value]=most_frequent_class
        errors.append(error)
    total_error=sum(errors)
    return predictors,total_error

In [22]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_d,y,random_state=14)

all_predictors={}
errors={}
for feature_index in range(X_train.shape[1]):    #shape shows (nrows,ncolumns), so shape[1] means ncolumns
    predictors,total_error=train_on_feature(X_train,y_train,feature_index)
    all_predictors[feature_index]=predictors
    errors[feature_index]=total_error

best_feature,best_error =sorted(errors.items(),key=itemgetter(1))[0]
model ={'feature':best_feature,'predictor':all_predictors[best_feature]}

def predict(X_test,model):
    variable=model['feature']
    predictor=model['predictor']
    y_predicted=np.array([predictor[int(sample[variable])] for sample in X_test ])
    return y_predicted
y_predicted=predict(X_test,model)
accuracy=np.mean(y_predicted==y_test)*100
print('the test accuracy is {:.1f}%'.format(accuracy))

In [ ]:
#question : so the feature{0,1} can predict target{0,1,2} with the accuray 65.8% ,incredible